In [314]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns 
from datetime import datetime, timedelta
from sklearn.feature_selection import chi2
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix
import seaborn as sns

# ROC AUC
Dummy for SPN


In [22]:
faults = pd.read_pickle('../data/tomo/faults_filtered.pkl')
y_derate = pd.read_pickle('../data/tomo/target_derate.pkl')
y_75derate = pd.read_pickle('../data/tomo/target_75derate.pkl')
diagnostics_imputed = pd.read_pickle('../data/tomo/diagnostics_imputed.pkl')

,RecordID,target
536775,1248457,0
443249,1248456,0
505537,1248455,0
463548,1248452,0
453676,1248446,0
...,...,...
546668,20,0
546669,21,0
546322,15,0
52370,1,0


In [6]:
df_5246 =  pd.read_csv('../cleanedData/faults_all_5246_6Hours.csv', low_memory=False) 

In [7]:
df_5246

,Unnamed: 0,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
0,0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,...,30525.608953,0.673641,109.4,131.0,17407.0,0.000000,3276.75,0.0,0.29,0
1,1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,...,30476.340865,0.581180,123.8,158.0,17407.0,7.854520,3276.75,0.0,0.00,0
2,2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,...,51537.457711,1.122735,116.6,93.2,17407.0,4.757373,3276.75,0.0,1.45,0
3,3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,...,51537.457711,0.000000,116.6,93.2,17407.0,0.000000,3276.75,0.0,0.00,0
4,4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,...,63434.313986,0.541554,80.6,93.2,17407.0,12.733260,3276.75,0.0,0.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,...,1004.910486,6.036350,NaN,84.2,50175.0,158.447200,13.95,100.0,6.38,0
546660,546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,...,2662.590112,1.862419,NaN,55.4,2047.0,3.126274,13.90,100.0,1.45,0
546661,546661,1248457,2020-03-06 14:14:13,111,18,True,8.0,2377,2020-03-06,2020-03-06,...,4042.492826,0.000000,NaN,100.4,1023.0,11.844890,14.10,100.0,1.74,0
546662,546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,...,65.514669,0.607598,NaN,118.4,1023.0,0.000000,13.95,100.0,0.29,0


In [9]:
df_5246 =  df_5246.drop(columns=['Unnamed: 0'])

In [11]:
# for right now, for the trucks that didn't have values, just use average from other trucks
df_5246['AcceleratorPedal'] = df_5246['AcceleratorPedal'].fillna(value=df_5246['AcceleratorPedal'].mean())
df_5246['CruiseControlSetSpeed'] = df_5246['CruiseControlSetSpeed'].fillna(value=df_5246['CruiseControlSetSpeed'].mean())
df_5246['EngineTimeLtd'] = df_5246['EngineTimeLtd'].fillna(value=df_5246['EngineTimeLtd'].mean())
df_5246['FuelLevel'] = df_5246['FuelLevel'].fillna(value=df_5246['FuelLevel'].mean())
df_5246['FuelTemperature'] = df_5246['FuelTemperature'].fillna(value=df_5246['FuelTemperature'].mean())
df_5246['SwitchedBatteryVoltage'] = df_5246['SwitchedBatteryVoltage'].fillna(value=df_5246['SwitchedBatteryVoltage'].mean())
df_5246['Throttle'] = df_5246['Throttle'].fillna(value=df_5246['Throttle'].mean())

In [12]:
df_5246

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,480159,...,30525.608953,0.673641,109.400000,131.0,17407.0,0.000000,3276.75,0.0,0.29,0
1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,477361,...,30476.340865,0.581180,123.800000,158.0,17407.0,7.854520,3276.75,0.0,0.00,0
2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,1059736,...,51537.457711,1.122735,116.600000,93.2,17407.0,4.757373,3276.75,0.0,1.45,0
3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,1059737,...,51537.457711,0.000000,116.600000,93.2,17407.0,0.000000,3276.75,0.0,0.00,0
4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,1219096,...,63434.313986,0.541554,80.600000,93.2,17407.0,12.733260,3276.75,0.0,0.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,1232638,...,1004.910486,6.036350,36.499619,84.2,50175.0,158.447200,13.95,100.0,6.38,0
546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,1241225,...,2662.590112,1.862419,36.499619,55.4,2047.0,3.126274,13.90,100.0,1.45,0
546661,1248457,2020-03-06 14:14:13,111,18,True,8.0,2377,2020-03-06,2020-03-06,1248457,...,4042.492826,0.000000,36.499619,100.4,1023.0,11.844890,14.10,100.0,1.74,0
546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,1247712,...,65.514669,0.607598,36.499619,118.4,1023.0,0.000000,13.95,100.0,0.29,0


In [ ]:
df_5246[df_5246['tar']]

In [14]:
df_5246.isna().sum()

RecordID                      0
EventTimeStamp                0
spn                           0
fmi                           0
active                        0
activeTransitionCount         0
EquipmentID                   0
EventTimeStamp_DateOnly       0
LocationTimeStamp_DateOnly    0
FaultId                       0
AcceleratorPedal              0
BarometricPressure            0
CruiseControlSetSpeed         0
DistanceLtd                   0
EngineCoolantTemperature      0
EngineLoad                    0
EngineOilPressure             0
EngineOilTemperature          0
EngineRpm                     0
EngineTimeLtd                 0
FuelLevel                     0
FuelLtd                       0
FuelRate                      0
FuelTemperature               0
IntakeManifoldTemperature     0
LampStatus                    0
Speed                         0
SwitchedBatteryVoltage        0
Throttle                      0
TurboBoostPressure            0
target                        0
dtype: i

## Use df_5246 to split data 80-20 based on Equipment ID split 80-20

In [ ]:
#total_rows = len(df)
#percentage = 0.5  # example percentage to filter on
#num_rows_to_keep = int(total_rows * percentage)
#filtered_df = df.iloc[:num_rows_to_keep, :]


In [28]:
# faults_all_5246_6Hours.csv use this file to train and split for full derate
df_5246 = pd.read_csv('../cleanedData/faults_all_5246_6Hours.csv', low_memory=False)
df_5246

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,480159,...,30525.608953,0.673641,109.4,131.0,17407.0,0.000000,3276.75,0.0,0.29,0
1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,477361,...,30476.340865,0.581180,123.8,158.0,17407.0,7.854520,3276.75,0.0,0.00,0
2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,1059736,...,51537.457711,1.122735,116.6,93.2,17407.0,4.757373,3276.75,0.0,1.45,0
3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,1059737,...,51537.457711,0.000000,116.6,93.2,17407.0,0.000000,3276.75,0.0,0.00,0
4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,1219096,...,63434.313986,0.541554,80.6,93.2,17407.0,12.733260,3276.75,0.0,0.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,1232638,...,1004.910486,6.036350,NaN,84.2,50175.0,158.447200,13.95,100.0,6.38,0
546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,1241225,...,2662.590112,1.862419,NaN,55.4,2047.0,3.126274,13.90,100.0,1.45,0
546661,1248457,2020-03-06 14:14:13,111,18,True,8.0,2377,2020-03-06,2020-03-06,1248457,...,4042.492826,0.000000,NaN,100.4,1023.0,11.844890,14.10,100.0,1.74,0
546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,1247712,...,65.514669,0.607598,NaN,118.4,1023.0,0.000000,13.95,100.0,0.29,0


In [30]:
# for right now, for the trucks that didn't have values, just use average from other trucks
df_5246['AcceleratorPedal'] = df_5246['AcceleratorPedal'].fillna(value=df_5246['AcceleratorPedal'].mean())
df_5246['CruiseControlSetSpeed'] = df_5246['CruiseControlSetSpeed'].fillna(value=df_5246['CruiseControlSetSpeed'].mean())
df_5246['EngineTimeLtd'] = df_5246['EngineTimeLtd'].fillna(value=df_5246['EngineTimeLtd'].mean())
df_5246['FuelLevel'] = df_5246['FuelLevel'].fillna(value=df_5246['FuelLevel'].mean())
df_5246['FuelTemperature'] = df_5246['FuelTemperature'].fillna(value=df_5246['FuelTemperature'].mean())
df_5246['SwitchedBatteryVoltage'] = df_5246['SwitchedBatteryVoltage'].fillna(value=df_5246['SwitchedBatteryVoltage'].mean())
df_5246['Throttle'] = df_5246['Throttle'].fillna(value=df_5246['Throttle'].mean())

In [31]:
df_5246.isna().sum()

RecordID                      0
EventTimeStamp                0
spn                           0
fmi                           0
active                        0
activeTransitionCount         0
EquipmentID                   0
EventTimeStamp_DateOnly       0
LocationTimeStamp_DateOnly    0
FaultId                       0
AcceleratorPedal              0
BarometricPressure            0
CruiseControlSetSpeed         0
DistanceLtd                   0
EngineCoolantTemperature      0
EngineLoad                    0
EngineOilPressure             0
EngineOilTemperature          0
EngineRpm                     0
EngineTimeLtd                 0
FuelLevel                     0
FuelLtd                       0
FuelRate                      0
FuelTemperature               0
IntakeManifoldTemperature     0
LampStatus                    0
Speed                         0
SwitchedBatteryVoltage        0
Throttle                      0
TurboBoostPressure            0
target                        0
dtype: i

## Use df_5246 to split data based on Equipment ID 

In [229]:
#Function to get X,y from the dataframe
#1. Parameters
#- dataframe df_5246
# - spn to split data
# -percentage

def test_split_data_custom(df, spn, percentage):
    #set parameters
   # df = df_5246
    #spn = 5246
    #percentage = 0.8  # example percentage to filter on

    all_Equip = df['EquipmentID'].unique()
    derate_Equip = df.loc[df['spn'] == spn]['EquipmentID'].unique()
    no_derate_Equip= all_Equip[np.isin(all_Equip, derate_Equip, invert=True)]

    print(len(all_Equip))
    print(len(derate_Equip))
    print(len(no_derate_Equip))

    #Commented this because we won't get same results everytime we run this
    #shuffle the array so that we get ramdom sequence in sample split
    #all_Equip_s = random.sample(all_Equip.tolist(), len(all_Equip))
    #derate_Equip_s = random.sample(derate_Equip.tolist(), len(derate_Equip))
    #no_derate_Equip_s = random.sample(no_derate_Equip.tolist(), len(no_derate_Equip))

    #shuffle(sklearn) the array so that we get ramdom sequence in sample split
    all_Equip_s = shuffle(all_Equip, random_state=42)
    derate_Equip_s = shuffle(derate_Equip, random_state=42)
    no_derate_Equip_s = shuffle(no_derate_Equip, random_state=42)


    print(len(all_Equip_s))
    print(len(derate_Equip_s))
    print(len(no_derate_Equip_s))

    #convert to dataframe to locate rows
    df_all_Equip = pd.DataFrame(all_Equip_s, columns = ['EquipmentID'])
    df_derate_Equip = pd.DataFrame(derate_Equip_s, columns = ['EquipmentID'])
    df_no_derate_Equip = pd.DataFrame(no_derate_Equip_s, columns = ['EquipmentID'])


    #get the equipments % based on the passed percentage
    #keep_rows = int(total_rows * percentage)
    keep_rows_nde = int(len(no_derate_Equip_s) * percentage)
    print(keep_rows_nde)

    #============================================
    #STEP- 4 Get X1 for no_derate_Equip
    #============================================
    df_nde_pct_X1_train = df_no_derate_Equip.iloc[:keep_rows_nde, :]
    print(df_nde_pct_X1_train)

    #get rest of the percentage by doing not isin  lookup
    df_nde_pct_X1_test = df_no_derate_Equip[np.isin(df_no_derate_Equip,df_nde_pct_X1_train, invert=True)]
    print(df_nde_pct_X1_test) 

    #Make sure data is correct by checking we dont have equip id in both dataframes
    should_be_zero_nde= df_nde_pct_X1_train[np.isin(df_nde_pct_X1_train,df_nde_pct_X1_test)]
    print("should_be_zero nde = "  + str(should_be_zero_nde.size)) 

    #============================================
    #STEP 5 get X2 or train and Test
    #============================================
    keep_rows_de = int(len(derate_Equip_s) * percentage)
    print(keep_rows_de)

    df_de_pct_X2_train = df_derate_Equip.iloc[:keep_rows_de, :]
    print(df_de_pct_X2_train)

    #get rest of the percentage by doing not isin  lookup
    df_de_pct_X2_test = df_derate_Equip[np.isin(df_derate_Equip,df_de_pct_X2_train, invert=True)]
    print(df_de_pct_X2_test) 

    #Make sure data is correct by checking we dont have equip id in both dataframes
    should_be_zero_de= df_de_pct_X2_train[np.isin(df_de_pct_X2_train,df_de_pct_X2_test)]
    print("should_be_zero derate = "  + str(should_be_zero_de.size)) 


    # now we have the id ready for trian and test from both derate and no derate data frames. 
    # combine df_nde_pct_X1_train & df_de_pct_X2_train, combine df_nde_pct_X1_test and df_de_pct_X2_test
    combined_df_X_train = pd.concat([df_nde_pct_X1_train, df_de_pct_X2_train], ignore_index=True)
    print(combined_df_X_train)


    # combine df_nde_pct_X1_test & df_de_pct_X2_test, combine df_de_pct_X2_test and df_de_pct_X2_test
    combined_df_X_test = pd.concat([df_nde_pct_X1_test, df_de_pct_X2_test], ignore_index=True)
    print(combined_df_X_test)



    #STEP 6 - Get X_train, y_train and X_test. y_test 
    # Next filter the data from original dataframe for X, y

    df_train = df[df['EquipmentID'].isin(combined_df_X_train['EquipmentID'].tolist())]
    X_train= df_train.drop(columns=['target'])
    y_train = df_train['target']

    df_test = df[df['EquipmentID'].isin(combined_df_X_test['EquipmentID'].tolist())]
    X_test =df_test.drop(columns=['target'])
    y_test = df_test['target']

   # x_train, x_test, y_train, y_test

    return X_train, X_test, y_train, y_test




In [284]:
X_train, X_test, y_train, y_test = test_split_data_custom(df_5246, 5246, 0.8)

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
418,204627,2015-09-15 15:21:07,5444,1,True,126.0,302,2015-09-15,2015-09-15,204627,...,62.827245,29255.097784,1.515400,89.937874,107.203941,17407.0,10.817993,3276.75,5.962416,2.048694
419,255258,2015-10-26 07:15:53,829,3,True,126.0,302,2015-10-26,2015-10-26,255258,...,77.200000,25167.010964,0.607598,87.800000,138.200000,5375.0,0.000000,3276.75,0.000000,0.290000
420,113208,2015-07-07 16:02:33,5444,1,True,126.0,302,2015-07-07,2015-07-07,113208,...,62.827245,29255.097784,1.515400,89.937874,107.203941,17407.0,10.817993,3276.75,5.962416,2.048694
421,996208,2018-04-10 03:10:57,111,18,True,48.0,302,2018-04-10,2018-04-10,996208,...,79.600000,38710.319554,1.426534,50.000000,50.000000,17407.0,0.000000,3276.75,0.000000,0.580000
422,96260,2015-06-23 07:41:20,5444,1,True,126.0,302,2015-06-23,2015-06-23,96260,...,62.827245,29255.097784,1.515400,89.937874,107.203941,17407.0,10.817993,3276.75,5.962416,2.048694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546657,1246549,2020-02-28 04:56:18,111,18,True,4.0,2377,2020-02-28,2020-02-28,1246549,...,99.200000,3604.231391,1.149152,36.499619,59.000000,2047.0,0.000000,14.00,100.000000,0.290000
546658,1232624,2020-01-14 13:42:40,1231,8,True,1.0,2377,2020-01-14,2020-01-14,1232624,...,3.600000,1004.646314,0.660432,36.499619,140.000000,50175.0,158.447200,13.85,100.000000,0.000000
546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,1232638,...,8.400001,1004.910486,6.036350,36.499619,84.200000,50175.0,158.447200,13.95,100.000000,6.380000
546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,1241225,...,66.800000,2662.590112,1.862419,36.499619,55.400000,2047.0,3.126274,13.90,100.000000,1.450000


In [285]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
546651    0
546652    0
546653    0
546662    0
546663    0
Name: target, Length: 453023, dtype: int64

In [288]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
cols_to_drop = ['RecordID', 'EquipmentID','EventTimeStamp', 'EventTimeStamp_DateOnly',  'LocationTimeStamp_DateOnly']

pipe = Pipeline(
    steps = [       
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression(solver='lbfgs',max_iter=10000))
    ]
)
pipe.fit(X_train.drop(columns=cols_to_drop), y_train)
y_pred = pipe.predict(X_test.drop(columns=cols_to_drop))
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [278]:
logreg = LogisticRegression(solver='lbfgs',max_iter=800000)
cols_to_drop = ['RecordID', 'EquipmentID','EventTimeStamp', 'EventTimeStamp_DateOnly',  'LocationTimeStamp_DateOnly']
df_x = X_train.drop(columns=cols_to_drop)
# Fit the classifier to the training data
#X_train, X_test, y_train, y_test 
logreg.fit(df_x, y_train)
y_pred = logreg.predict(X_test.drop(columns=cols_to_drop))


C:\Users\kalea\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [274]:
y_pred

Index(['RecordID', 'EventTimeStamp', 'spn', 'fmi', 'active',
       'activeTransitionCount', 'EquipmentID', 'EventTimeStamp_DateOnly',
       'LocationTimeStamp_DateOnly', 'FaultId', 'AcceleratorPedal',
       'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature',
       'LampStatus', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure'],
      dtype='object')

In [241]:
X_train.drop(columns=['EventTimeStamp', 'EventTimeStamp_DateOnly', 'LocationTimeStamp', 'LocationTimeStamp_DateOnly'])

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,480159,...,73.2,30525.608953,0.673641,109.400000,131.0,17407.0,0.000000,3276.75,0.0,0.29
1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,477361,...,45.6,30476.340865,0.581180,123.800000,158.0,17407.0,7.854520,3276.75,0.0,0.00
2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,1059736,...,76.4,51537.457711,1.122735,116.600000,93.2,17407.0,4.757373,3276.75,0.0,1.45
3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,1059737,...,71.6,51537.457711,0.000000,116.600000,93.2,17407.0,0.000000,3276.75,0.0,0.00
4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,1219096,...,62.8,63434.313986,0.541554,80.600000,93.2,17407.0,12.733260,3276.75,0.0,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546651,1236474,2020-01-24 11:41:32,111,18,True,9.0,2376,2020-01-24,2020-01-24,1236474,...,82.0,1806.408492,18.571350,36.499619,96.8,2047.0,35.029800,14.00,100.0,26.97
546652,1237058,2020-01-27 12:06:07,111,18,True,28.0,2376,2020-01-27,2020-01-27,1237058,...,54.8,1953.023980,0.000000,36.499619,48.2,2047.0,57.204990,14.00,28.8,0.00
546653,1236460,2020-01-24 11:00:52,111,18,True,6.0,2376,2020-01-24,2020-01-24,1236460,...,75.6,1802.974255,2.905902,36.499619,78.8,2047.0,67.865390,14.05,100.0,3.19
546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,1247712,...,32.0,65.514669,0.607598,36.499619,118.4,1023.0,0.000000,13.95,100.0,0.29


# Using SMOTE

In [290]:
df_1569 =  pd.read_csv('../cleanedData/faults_all_1569_6Hours.csv', low_memory=False) 

df_1569['AcceleratorPedal'] = df_1569['AcceleratorPedal'].fillna(value=df_1569['AcceleratorPedal'].mean())
df_1569['CruiseControlSetSpeed'] = df_1569['CruiseControlSetSpeed'].fillna(value=df_1569['CruiseControlSetSpeed'].mean())
df_1569['EngineTimeLtd'] = df_1569['EngineTimeLtd'].fillna(value=df_1569['EngineTimeLtd'].mean())
df_1569['FuelLevel'] = df_1569['FuelLevel'].fillna(value=df_1569['FuelLevel'].mean())
df_1569['FuelTemperature'] = df_1569['FuelTemperature'].fillna(value=df_1569['FuelTemperature'].mean())
df_1569['SwitchedBatteryVoltage'] = df_1569['SwitchedBatteryVoltage'].fillna(value=df_1569['SwitchedBatteryVoltage'].mean())
df_1569['Throttle'] = df_1569['Throttle'].fillna(value=df_1569['Throttle'].mean())

X_train, X_test, y_train, y_test =  test_split_data_custom(df_1569, 1569, 0.8)


1039
491
548
1039
491
548
438
     EquipmentID
0           2069
1           1484
2           2067
3           2177
4           2305
..           ...
433         2314
434         1724
435         1454
436         1702
437         1678

[438 rows x 1 columns]
     EquipmentID
438         2361
439         1332
440         2149
441         2167
442         1459
..           ...
543         1471
544         1550
545         2092
546         2260
547         1543

[110 rows x 1 columns]
should_be_zero nde = 0
392
     EquipmentID
0           1977
1           1476
2           1956
3           2005
4           1949
..           ...
387         1766
388         1946
389         1940
390         1844
391         1704

[392 rows x 1 columns]
     EquipmentID
392         1660
393         1665
394         1790
395         1971
396         1720
..           ...
486         1512
487         1750
488         1856
489         1957
490         1506

[99 rows x 1 columns]
should_be_zero derate = 0
     E

In [321]:
print(X_train.shape[0])
print(y_train.shape[0])
print(X_test.shape[0])
print(y_test.shape[0])


453023
453023
93641
93641


In [320]:
df_1569

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target
0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,480159,...,30525.608953,0.673641,109.400000,131.0,17407.0,0.000000,3276.75,0.0,0.29,0
1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,477361,...,30476.340865,0.581180,123.800000,158.0,17407.0,7.854520,3276.75,0.0,0.00,0
2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,1059736,...,51537.457711,1.122735,116.600000,93.2,17407.0,4.757373,3276.75,0.0,1.45,0
3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,1059737,...,51537.457711,0.000000,116.600000,93.2,17407.0,0.000000,3276.75,0.0,0.00,0
4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,1219096,...,63434.313986,0.541554,80.600000,93.2,17407.0,12.733260,3276.75,0.0,0.29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,1232638,...,1004.910486,6.036350,36.499619,84.2,50175.0,158.447200,13.95,100.0,6.38,0
546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,1241225,...,2662.590112,1.862419,36.499619,55.4,2047.0,3.126274,13.90,100.0,1.45,0
546661,1248457,2020-03-06 14:14:13,111,18,True,8.0,2377,2020-03-06,2020-03-06,1248457,...,4042.492826,0.000000,36.499619,100.4,1023.0,11.844890,14.10,100.0,1.74,0
546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,1247712,...,65.514669,0.607598,36.499619,118.4,1023.0,0.000000,13.95,100.0,0.29,0


In [291]:
df_1569.isna().sum()

RecordID                      0
EventTimeStamp                0
spn                           0
fmi                           0
active                        0
activeTransitionCount         0
EquipmentID                   0
EventTimeStamp_DateOnly       0
LocationTimeStamp_DateOnly    0
FaultId                       0
AcceleratorPedal              0
BarometricPressure            0
CruiseControlSetSpeed         0
DistanceLtd                   0
EngineCoolantTemperature      0
EngineLoad                    0
EngineOilPressure             0
EngineOilTemperature          0
EngineRpm                     0
EngineTimeLtd                 0
FuelLevel                     0
FuelLtd                       0
FuelRate                      0
FuelTemperature               0
IntakeManifoldTemperature     0
LampStatus                    0
Speed                         0
SwitchedBatteryVoltage        0
Throttle                      0
TurboBoostPressure            0
target                        0
dtype: i

In [322]:
cols_to_drop = ['RecordID', 'EquipmentID','EventTimeStamp', 'EventTimeStamp_DateOnly',  'LocationTimeStamp_DateOnly']

oversampler = SMOTE(k_neighbors=5, random_state=42)
X_smote, y_smote = oversampler.fit_resample(X_train.drop(columns=cols_to_drop), y_train.drop(columns=cols_to_drop))

y_smote.value_counts()

0    452247
1    452247
Name: target, dtype: int64

In [299]:
gbr_smoted = Pipeline(
    steps = [
        #('scaler', MinMaxScaler()),
        ('gb', GradientBoostingClassifier(verbose=True))#n_estimators = 1000, learning_rate=0.01
    ]
)

gbr_smoted.fit(X_smote, y_smote)

      Iter       Train Loss   Remaining Time 
         1           1.2661            8.31m
         2           1.1690            8.39m
         3           1.0867            8.30m
         4           1.0188            8.08m
         5           0.9587            8.01m
         6           0.9002            7.93m
         7           0.8561            7.82m
         8           0.8116            7.77m
         9           0.7735            7.64m
        10           0.7429            7.52m
        20           0.5572            6.79m
        30           0.4681            5.89m
        40           0.4028            5.00m
        50           0.3550            4.14m
        60           0.3297            3.32m
        70           0.3070            2.53m
        80           0.2866            1.73m
        90           0.2676           52.57s
       100           0.2505            0.00s


Pipeline(steps=[('gb', GradientBoostingClassifier(verbose=True))])

In [306]:
y_predict = gbr_smoted.predict(X_test.drop(columns=cols_to_drop))
y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [305]:
confusion_matrix(y_test, gbr_smoted.predict(X_test.drop(columns=cols_to_drop)))

array([[88839,  4678],
       [   12,   112]], dtype=int64)

In [324]:
X_test.columns

Index(['RecordID', 'EventTimeStamp', 'spn', 'fmi', 'active',
       'activeTransitionCount', 'EquipmentID', 'EventTimeStamp_DateOnly',
       'LocationTimeStamp_DateOnly', 'FaultId', 'AcceleratorPedal',
       'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature',
       'LampStatus', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure'],
      dtype='object')

In [309]:
len(y_predict)

93641

In [310]:
len(y_test)

93641

In [312]:
dfTest = pd.DataFrame(list(zip(y_test, y_predict)),
                  columns=['Actual', 'Pred'])
dfTest.head

<bound method NDFrame.head of        Actual  Pred
0           0     0
1           0     0
2           0     0
3           0     0
4           0     0
...       ...   ...
93636       0     0
93637       0     0
93638       0     0
93639       0     0
93640       0     0

[93641 rows x 2 columns]>

In [313]:
dfTest.to_csv('../data/dfTest.csv', index=False)  

In [330]:
faults.columns

Index(['RecordID', 'EventTimeStamp', 'spn', 'fmi', 'active',
       'activeTransitionCount', 'EquipmentID', 'EventTimeStamp_DateOnly',
       'LocationTimeStamp_DateOnly', 'FaultId', 'AcceleratorPedal',
       'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature',
       'LampStatus', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure', 'target', 'spn_fmi'],
      dtype='object')

## Create Dummy features from spn_fmi

In [372]:
faults = df_1569

In [373]:
cols_to_drop = ['RecordID', 'EquipmentID','EventTimeStamp', 
                'EventTimeStamp_DateOnly',  'LocationTimeStamp_DateOnly',
               'active', 'FaultId','spn', 'fmi']

In [374]:
faults['spn_fmi'] = ['_'.join(i) for i in zip(faults['spn'].astype(str), faults['fmi'].astype(str))]
faults

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target,spn_fmi
0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,480159,...,0.673641,109.400000,131.0,17407.0,0.000000,3276.75,0.0,0.29,0,5396_1
1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,477361,...,0.581180,123.800000,158.0,17407.0,7.854520,3276.75,0.0,0.00,0,5396_1
2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,1059736,...,1.122735,116.600000,93.2,17407.0,4.757373,3276.75,0.0,1.45,0,3251_4
3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,1059737,...,0.000000,116.600000,93.2,17407.0,0.000000,3276.75,0.0,0.00,0,1668_19
4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,1219096,...,0.541554,80.600000,93.2,17407.0,12.733260,3276.75,0.0,0.29,0,649_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,1232638,...,6.036350,36.499619,84.2,50175.0,158.447200,13.95,100.0,6.38,0,1231_8
546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,1241225,...,1.862419,36.499619,55.4,2047.0,3.126274,13.90,100.0,1.45,0,111_18
546661,1248457,2020-03-06 14:14:13,111,18,True,8.0,2377,2020-03-06,2020-03-06,1248457,...,0.000000,36.499619,100.4,1023.0,11.844890,14.10,100.0,1.74,0,111_18
546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,1247712,...,0.607598,36.499619,118.4,1023.0,0.000000,13.95,100.0,0.29,0,1761_17


In [375]:
faults_encoded = pd.get_dummies(faults, columns=['spn_fmi'], prefix='spn_fmi')
#faults_encoded = faults_encoded.drop(columns=cols_to_drop)
faults_encoded

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,spn_fmi_96_3,spn_fmi_96_4,spn_fmi_96_9,spn_fmi_976_9,spn_fmi_97_15,spn_fmi_97_16,spn_fmi_97_3,spn_fmi_97_4,spn_fmi_98_18,spn_fmi_98_5
0,480159,2016-05-19 09:44:56,5396,1,True,94.0,301,2016-05-19,2016-05-19,480159,...,0,0,0,0,0,0,0,0,0,0
1,477361,2016-05-16 13:55:05,5396,1,True,91.0,301,2016-05-16,2016-05-16,477361,...,0,0,0,0,0,0,0,0,0,0
2,1059736,2018-09-24 14:28:35,3251,4,True,8.0,301,2018-09-24,2018-09-24,1059736,...,0,0,0,0,0,0,0,0,0,0
3,1059737,2018-09-24 14:34:28,1668,19,True,52.0,301,2018-09-24,2018-09-24,1059737,...,0,0,0,0,0,0,0,0,0,0
4,1219096,2019-11-23 07:30:05,649,2,True,1.0,301,2019-11-23,2019-11-23,1219096,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546659,1232638,2020-01-14 13:46:39,1231,8,True,2.0,2377,2020-01-14,2020-01-14,1232638,...,0,0,0,0,0,0,0,0,0,0
546660,1241225,2020-02-12 05:42:02,111,18,True,2.0,2377,2020-02-12,2020-02-12,1241225,...,0,0,0,0,0,0,0,0,0,0
546661,1248457,2020-03-06 14:14:13,111,18,True,8.0,2377,2020-03-06,2020-03-06,1248457,...,0,0,0,0,0,0,0,0,0,0
546662,1247712,2020-03-04 10:06:10,1761,17,True,1.0,2380,2020-03-04,2020-03-04,1247712,...,0,0,0,0,0,0,0,0,0,0


In [376]:
X_train, X_test, y_train, y_test =  test_split_data_custom(faults_encoded, 1569, 0.8)

1039
491
548
1039
491
548
438
     EquipmentID
0           2069
1           1484
2           2067
3           2177
4           2305
..           ...
433         2314
434         1724
435         1454
436         1702
437         1678

[438 rows x 1 columns]
     EquipmentID
438         2361
439         1332
440         2149
441         2167
442         1459
..           ...
543         1471
544         1550
545         2092
546         2260
547         1543

[110 rows x 1 columns]
should_be_zero nde = 0
392
     EquipmentID
0           1977
1           1476
2           1956
3           2005
4           1949
..           ...
387         1766
388         1946
389         1940
390         1844
391         1704

[392 rows x 1 columns]
     EquipmentID
392         1660
393         1665
394         1790
395         1971
396         1720
..           ...
486         1512
487         1750
488         1856
489         1957
490         1506

[99 rows x 1 columns]
should_be_zero derate = 0
     E

In [377]:
oversampler = SMOTE(k_neighbors=5, random_state=42)
X_smote, y_smote = oversampler.fit_resample(X_train.drop(columns=cols_to_drop), y_train.drop(columns=cols_to_drop))
y_smote.value_counts()

0    452247
1    452247
Name: target, dtype: int64

In [343]:
gbr_smoted = Pipeline(
    steps = [
        #('scaler', MinMaxScaler()),
        ('gb', GradientBoostingClassifier(verbose=True))#n_estimators = 1000, learning_rate=0.01
    ]
)

gbr_smoted.fit(X_smote, y_smote)

      Iter       Train Loss   Remaining Time 
         1           1.2662           24.23m
         2           1.1673           22.86m
         3           1.0853           22.22m
         4           1.0170           21.36m
         5           0.9594           20.84m
         6           0.9085           20.49m
         7           0.8654           20.15m
         8           0.8269           19.84m
         9           0.7859           19.59m
        10           0.7565           19.37m
        20           0.5601           17.04m
        30           0.4794           14.90m
        40           0.4366           13.00m
        50           0.4083           10.95m
        60           0.3887            8.81m
        70           0.3716            6.62m
        80           0.3563            4.42m
        90           0.3441            2.21m
       100           0.3371            0.00s


Pipeline(steps=[('gb', GradientBoostingClassifier(verbose=True))])

In [344]:
y_predict = gbr_smoted.predict(X_test.drop(columns=cols_to_drop))
y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [345]:
confusion_matrix(y_test, gbr_smoted.predict(X_test.drop(columns=cols_to_drop)))

array([[88303,  5214],
       [   11,   113]], dtype=int64)

In [349]:
df_1569[df_1569['target'] ==1]['spn'].unique()

array([  3556,   1787,   3821,   1668,   3251,   5848,   3246,   1761,
         3697,   4766,   4360,     74,   3226,   3361,   4349,   1231,
          639,   1483,   5742,   3216,   3217,   5743,   5109,   3698,
          649,   3610,   2791,    641,   1072,    171,    184,   4363,
          111,   5394,   1569,   4094,   4342,  50353,   3031,   3242,
          596,   4344,   4364,   3360,   3362,   1068,     97,    792,
          791,    158,    829,   4376, 523531,   4334,   3363,   3058,
          102,   5319,    629,    653,   4096,    105,   5485,     91,
          252,    245,    251,    228,   2017,     84,   1045,     94,
         4375,    627,   4340,    611,    789,    790,   5024,    929,
         6802,     96,   5396,   3364,    917,    976,   3480,   3482,
         1612,    609,   1028,    248,   2863,   3490,     27,   1347,
          630,    103,   2623,   5835,     70, 520953], dtype=int64)

In [352]:
df_spn = df_1569[df_1569['target'] ==1]
df_spn

,RecordID,EventTimeStamp,spn,fmi,active,activeTransitionCount,EquipmentID,EventTimeStamp_DateOnly,LocationTimeStamp_DateOnly,FaultId,...,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,target,spn_fmi
1208,1230352,2020-01-06 07:20:54,3556,5,True,1.0,302,2020-01-06,2020-01-06,1230352,...,0.541554,84.200000,93.2,18431.0,0.000000,3276.750000,0.0,0.58,1,3556_5
1220,1230351,2020-01-06 07:20:54,1787,11,True,1.0,302,2020-01-06,2020-01-06,1230351,...,0.541554,84.200000,93.2,18431.0,0.000000,3276.750000,0.0,0.58,1,1787_11
2764,383982,2016-02-15 10:59:20,3821,11,True,11.0,304,2016-02-15,2016-02-15,383982,...,0.528346,66.200000,55.4,18431.0,49.797080,3276.750000,3.6,5.51,1,3821_11
2770,383958,2016-02-15 10:50:42,1668,2,True,2.0,304,2016-02-15,2016-02-15,383958,...,6.445818,57.200000,75.2,17407.0,51.515560,3276.750000,29.6,4.64,1,1668_2
2783,383957,2016-02-15 10:50:39,3251,3,True,1.0,304,2016-02-15,2016-02-15,383957,...,4.015428,57.200000,77.0,17407.0,51.054380,3276.750000,18.4,2.03,1,3251_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543234,1085214,2018-11-27 09:22:18,3363,3,True,1.0,2209,2018-11-27,2018-11-27,1085214,...,0.000000,36.499619,53.6,18431.0,68.302280,3191.984572,28.8,0.00,1,3363_3
543244,1085251,2018-11-27 09:22:18,3363,3,True,1.0,2209,2018-11-27,2018-11-27,1085251,...,0.000000,36.499619,53.6,18431.0,68.302280,3191.984572,28.8,0.00,1,3363_3
543268,1238640,2020-02-03 10:37:54,5394,5,True,1.0,2211,2020-02-03,2020-02-03,1238640,...,1.003857,36.499619,68.0,17407.0,9.757469,3191.984572,100.0,0.29,1,5394_5
543269,1223046,2019-12-10 13:12:57,5394,5,True,3.0,2211,2019-12-10,2019-12-10,1223046,...,1.413325,36.499619,89.6,17407.0,2.776752,3191.984572,100.0,0.58,1,5394_5
